In [1]:
import pandas as pd

# Load the datasets
no_sp_df = pd.read_csv('../log/tmplog_finalTest4.csv')
sp_df = pd.read_csv('../log/tmplog_finalTest4_SP.csv')

# Display the first few rows of each DataFrame to understand the structure
no_sp_df.head(), sp_df.head()

(                       flow id:  first send:  last send:  last arrive:  \
 0  11:0:11 11:0:41 10099 100 17     2.015245   28.673368     28.742101   
 1  11:0:11 11:0:41 10097 100 17     2.014628   51.852337     51.852958   
 2  11:0:11 11:0:41 10096 100 17     2.014319   28.894661     28.928142   
 3  11:0:11 11:0:41 10093 100 17     2.013487   29.151326     29.172037   
 4  11:0:11 11:0:41 10092 100 17     2.013178   28.667184     28.736536   
 
    send num:  receive num:  send size:  receive size:       FCT:  origin_pg:  \
 0      11468         11520     1050000        1050000  26.726856           3   
 1      12683         12732     1049369        1049369  49.838330           3   
 2      12031         12085     1049895        1049895  26.913824           3   
 3      12135         12189     1049740        1049740  27.158550           3   
 4      11460         11512     1050000        1050000  26.723359           3   
 
    transfer delay:  
 0                0  
 1              

In [2]:
# Merge the datasets on 'flow id:' column
merged_df = pd.merge(no_sp_df, sp_df, on='flow id:', suffixes=('_no_sp', '_sp'))

# Calculate FCT change and percentage change
merged_df['FCT Change'] = merged_df['FCT:_sp'] - merged_df['FCT:_no_sp']
merged_df['FCT % Change'] = (merged_df['FCT Change'] / merged_df['FCT:_no_sp']) * 100

# Display the new dataframe with changes
merged_df[['flow id:', 'FCT:_no_sp', 'FCT:_sp', 'FCT Change', 'FCT % Change']].head()


,flow id:,FCT:_no_sp,FCT:_sp,FCT Change,FCT % Change
0,11:0:11 11:0:41 10099 100 17,26.726856,27.030868,0.304013,1.137481
1,11:0:11 11:0:41 10097 100 17,49.838330,50.360877,0.522547,1.048485
2,11:0:11 11:0:41 10096 100 17,26.913824,27.181101,0.267278,0.993087
3,11:0:11 11:0:41 10093 100 17,27.158550,27.362442,0.203893,0.750749
4,11:0:11 11:0:41 10092 100 17,26.723359,26.941711,0.218353,0.817085


In [3]:
# merged_df = pd.read_csv('./result_multiCast_BestWithOneChannelAllowed.csv')
# merged_df = pd.read_csv('./result_4.csv')
merged_df

,flow id:,first send:_no_sp,last send:_no_sp,last arrive:_no_sp,send num:_no_sp,receive num:_no_sp,send size:_no_sp,receive size:_no_sp,FCT:_no_sp,origin_pg:_no_sp,...,last arrive:_sp,send num:_sp,receive num:_sp,send size:_sp,receive size:_sp,FCT:_sp,origin_pg:_sp,transfer delay:_sp,FCT Change,FCT % Change
0,11:0:11 11:0:41 10099 100 17,2.015245,28.673368,28.742101,11468,11520,1050000,1050000,26.726856,3,...,29.046113,12013,12066,1053000,1053000,27.030868,3,0,0.304013,1.137481
1,11:0:11 11:0:41 10097 100 17,2.014628,51.852337,51.852958,12683,12732,1049369,1049369,49.838330,3,...,52.375505,13274,13324,1052369,1052369,50.360877,3,0,0.522547,1.048485
2,11:0:11 11:0:41 10096 100 17,2.014319,28.894661,28.928142,12031,12085,1049895,1049895,26.913824,3,...,29.195420,12567,12621,1049895,1049895,27.181101,3,0,0.267278,0.993087
3,11:0:11 11:0:41 10093 100 17,2.013487,29.151326,29.172037,12135,12189,1049740,1049740,27.158550,3,...,29.375929,12652,12705,1049740,1049740,27.362442,3,0,0.203893,0.750749
4,11:0:11 11:0:41 10092 100 17,2.013178,28.667184,28.736536,11460,11512,1050000,1050000,26.723359,3,...,28.954889,11925,11978,1053000,1053000,26.941711,3,0,0.218353,0.817085
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
295,11:0:31 11:0:41 10053 100 17,2.003374,5.357717,5.440893,774,788,66038,66038,3.437518,2,...,6.321055,831,864,66038,66038,4.317680,2,0,0.880162,25.604580
296,11:0:31 11:0:41 10054 100 17,2.003587,5.293382,5.376887,832,849,66072,66072,3.373300,2,...,6.309072,896,926,66072,66072,4.305485,2,0,0.932185,27.634206
297,11:0:31 11:0:41 10008 100 17,2.000392,3.538297,3.636610,132,134,8208,8208,1.636219,1,...,3.636610,132,134,8208,8208,1.636219,1,0,0.000000,0.000000
298,11:0:31 11:0:41 10058 100 17,2.004705,28.646076,28.717307,11444,11496,1050000,1050000,26.712602,3,...,29.025262,11993,12047,1053000,1053000,27.020557,3,0,0.307955,1.152845


In [4]:
# Group by origin priority group and calculate the required statistics
group_stats = merged_df.groupby('origin_pg:_no_sp').agg(
    FCT_Total_No_SP=pd.NamedAgg(column='FCT:_no_sp', aggfunc='sum'),
    FCT_Total_SP=pd.NamedAgg(column='FCT:_sp', aggfunc='sum'),
    Total_FCT_Change=pd.NamedAgg(column='FCT Change', aggfunc='sum'),
    Mean_FCT_Change=pd.NamedAgg(column='FCT % Change', aggfunc='mean'),
    Mean_FCT_Per_Change=pd.NamedAgg(column='FCT Change', aggfunc='mean'),
    Count_FCT_Decrease=pd.NamedAgg(column='FCT Change', aggfunc=lambda x: (x <= 0.0).sum()),
    Total_Flows=pd.NamedAgg(column='FCT Change', aggfunc='size')
)

# Calculate percentage of flows with decreased FCT
group_stats['Percentage_FCT_Decrease'] = (group_stats['Count_FCT_Decrease'] / group_stats['Total_Flows']) * 100

# Display the summarized statistics
group_stats.reset_index()


,origin_pg:_no_sp,FCT_Total_No_SP,FCT_Total_SP,Total_FCT_Change,Mean_FCT_Change,Mean_FCT_Per_Change,Count_FCT_Decrease,Total_Flows,Percentage_FCT_Decrease
0,1,241.653060,243.184125,1.531065,0.555178,0.011341,118,135,87.407407
1,2,120.169368,138.917269,18.747901,21.006642,0.624930,6,30,20.000000
2,3,3668.339438,3539.401589,-128.937849,-2.667274,-0.955095,32,135,23.703704


In [5]:
# merged_df.to_csv("./result_4.csv")

In [6]:
#0 	1 	15470.217402 	15470.217336 	-0.000066 	-4.888741e-07 	1 	135 	0.740741

In [7]:
# in 40Mpbs some high pg flows' fct has increase as follow: 
    # 131,11:0:21 11:0:41 10020 100 17,2.00055418,125.34508296,125.364080097,2083,2086,123.363525917,1,0,2.00055418,125.366682448,125.366845447,2086,2086,123.366291267,1,0,0.0027653499999900077,0.002241626914790485
        # use high_60 replace high_61
    # 156,11:0:21 11:0:41 10004 100 17,2.000111196,113.565199616,113.58213076,1627,1629,111.582019564,1,0,2.000111196,113.610388046,113.610699046,1629,1629,111.61058785,1,0,0.02856828599999517,0.02560294760000224
        # use high_12 replace high_13
    # 183,11:0:21 11:0:41 10040 100 17,2.001132561,120.484628618,120.506937341,1108,1111,118.50580478,1,0,2.001132561,120.556739024,120.557038222,1113,1114,118.555905661,1,0,0.05010088099999166,0.04227715350568809
        # use high_120 replace high_121
    # 180,11:0:21 11:0:41 10029 100 17,2.000844572,121.428252899,121.447419428,1951,1951,119.446574856,1,0,2.000844572,121.44740644,121.447481839,1951,1951,119.446637267,1,0,6.241100000181632e-05,5.225013783530965e-05
        # use high_87 replace high_88   
    # 256,11:0:31 11:0:41 10022 100 17,2.000626577,123.632155519,123.651544052,2069,2082,121.650917475,1,0,2.000626577,123.653212468,123.653460069,2069,2069,121.652833492,1,0,0.001916016999999215,0.0015750123712737045
        # use high_67 replace high_68       
    # 50,11:0:11 11:0:41 10020 100 17,2.00054058,124.004466398,124.021815529,2071,2075,122.021274949,1,0,2.00054058,124.023811895,124.023887895,2071,2071,122.023347315,1,0,0.0020723659999930533,0.0016983644867333333
        # use high_59 replace high_60
        # use high_60 replace high_61
